In [83]:
# import tensorflow as tf
# print(tf.__version__)

2.2.1


In [ ]:
import tensorflow as tf
print(tf.__version__)

In [10]:
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.keras.utils import data_utils
from tensorflow.python.keras.utils import losses_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras import layers

In [2]:
# Load data from CSV file
data = pd.read_csv('./dataset/data.csv')

In [3]:
# Extract images, labels, and bounding boxes
images = data['filename'].tolist()
labels = data['class'].tolist()
boxes = data[['xmin', 'ymin', 'xmax', 'ymax']].values.tolist()

In [4]:
# Split data into training and validation sets
train_images, val_images, train_labels, val_labels, train_boxes, val_boxes = train_test_split(images, labels, boxes, test_size=0.2)

In [5]:
# Define data generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   rotation_range=30,
                                   zoom_range=[0.5, 1.5],
                                   width_shift_range=0.1,
                                   height_shift_range=0.1)

val_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, images, labels, boxes, batch_size=32, datagen=None, target_size=(224, 224)):
        self.images = images
        self.labels = labels
        self.boxes = boxes
        self.batch_size = batch_size
        self.datagen = datagen
        self.target_size = target_size

    def __len__(self):
        return int(np.ceil(len(self.images) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_images = self.images[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_boxes = self.boxes[idx * self.batch_size:(idx + 1) * self.batch_size]

        # Load images and resize to target size
        images = []
        for image_path in batch_images:
            image = load_img(image_path, target_size=self.target_size)
            image = img_to_array(image)
            images.append(image)
        images = np.array(images)

        # Apply data augmentation if specified
        if self.datagen:
            images = self.datagen.flow(images, shuffle=False).next()

        return images, batch_labels, batch_boxes


In [7]:
def loss_fn(boxes_true, boxes_pred):
    """Compute the mean squared error between true and predicted bounding boxes."""
    loss = tf.reduce_mean(tf.square(boxes_true - boxes_pred))
    return loss


In [8]:
train_generator = DataGenerator(train_images, train_labels, train_boxes, batch_size=32, datagen=train_datagen, target_size=(224, 224))
val_generator = DataGenerator(val_images, val_labels, val_boxes, batch_size=32, datagen=val_datagen, target_size=(224, 224))


In [12]:
# Load ResNet50 model without the top layer
# Load ResNet50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(4, activation='softmax', name='class_output')(x)

# Add custom bounding box regression layers
y = base_model.output
y = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(y)
y = layers.MaxPooling2D(pool_size=(2, 2))(y)
y = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(y)
y = layers.MaxPooling2D(pool_size=(2, 2))(y)
y = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(y)
y = layers.MaxPooling2D(pool_size=(1, 1))(y)
y = layers.Flatten()(y)
y = layers.Dense(512, activation='relu')(y)
y = layers.Dropout(0.5)(y)
y = layers.Dense(4, name='bbox_output')(y)


# Create a new model
model = Model(inputs=base_model.input, outputs=[predictions, y])

# Compile the model
model.compile(optimizer='adam',
              loss={'class_output': 'categorical_crossentropy', 'bbox_output': 'mse'},
              loss_weights={'class_output': 1., 'bbox_output': 1.})

In [13]:
# Compile model with Adam optimizer and Mean Squared Error loss
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='mse')

In [14]:
# Define training loop with AMP
opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(optimizer)

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [41]:
# Define training loop
epochs = 10
for epoch in range(epochs):
    x, (y_class, y_box) = batch
    # Train on one epoch
    for batch in train_generator:
        # Use mixed precision for improved performance on NVIDIA GPUs with Tensor Cores
        with tf.train.experimental.enable_mixed_precision():
            # Compute loss and metric values
            loss = model.train_on_batch(x, {'class_output': y_class, 'bbox_output': y_box})
            print('Batch loss:', loss)
            # Print loss and metric values every 100 steps
            if train_generator.total_batches_seen % 100 == 0:
                print('Epoch {}, Batch {}: Loss = {:.4f}'.format(epoch, train_generator.total_batches_seen, loss))
    # Evaluate on validation set
    val_loss = 0
    for x, (y_class, y_box) in val_generator:
        with tf.train.experimental.enable_mixed_precision():
            val_loss += model.test_on_batch(x, {'class_output': y_class, 'bbox_output': y_box})
    val_loss /= val_generator.total_batches_seen
    print('Epoch {}: Validation Loss = {:.4f}'.format(epoch, val_loss))

TypeError: cannot unpack non-iterable int object

In [17]:
test_loss = 0
y_true = []
y_pred = []
for batch in range(len(val_generator)):
    images, labels, boxes = test_generator[batch]
    predictions = model(images, training=False)
    loss = loss_fn(boxes, predictions)
    test_loss += loss.numpy()
    y_true.append(boxes.numpy())
    y_pred.append(predictions.numpy())
test_loss /= len(test_generator)

y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)

# Calculate evaluation metrics
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score
from tensorflow.keras.backend import binary_crossentropy

# Convert bounding boxes to classification labels
y_true_labels = np.zeros(y_true.shape[0])
y_pred_labels = np.zeros(y_pred.shape[0])
for i in range(y_true.shape[0]):
    if np.any(y_true[i] > 0):
        y_true_labels[i] = 1
    if np.any(y_pred[i] > 0):
        y_pred_labels[i] = 1

# Calculate classification metrics
confusion = confusion_matrix(y_true_labels, y_pred_labels)
f1 = f1_score(y_true_labels, y_pred_labels)
recall = recall_score(y_true_labels, y_pred_labels)
precision = precision_score(y_true_labels, y_pred_labels)

# Calculate regression metrics
iou = 0
mse = 0
for i in range(y_true.shape[0]):
    iou += intersection_over_union(y_true[i], y_pred[i])
    mse += mean_squared_error(y_true[i], y_pred[i])
iou /= y_true.shape[0]
mse /= y_true.shape[0]

print("Test loss:", test_loss)
print("Confusion matrix:\n", confusion)
print("F1 score:", f1)
print("Recall:", recall)
print("Precision:", precision)
print("Intersection over union:", iou)
print("Mean squared error:", mse)


NameError: name 'test_generator' is not defined

In [23]:
# Define the image data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [37]:
# Define the train generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filename",
    y_col=["class", "xmin", "ymin", "xmax", "ymax"],
    target_size=(224, 224),
    batch_size=32,
    class_mode="raw",
    subset="training",
    shuffle=True,
    seed=42,
    dtype='float32')

val_generator = val_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': val_images, 'class': val_labels, 'boxes': val_boxes}),
    x_col='filename',
    y_col=['class', 'boxes'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='other')

Found 1326 validated image filenames.
Found 415 validated image filenames.


In [38]:
# Load data from CSV file
data = pd.read_csv('./dataset/data.csv')

# Extract images, labels, and bounding boxes
images = data['filename'].tolist()
labels = data['class'].tolist()
boxes = data[['xmin', 'ymin', 'xmax', 'ymax']].values.tolist()

# Split data into training and validation sets
train_images, val_images, train_labels, val_labels, train_boxes, val_boxes = train_test_split(images, labels, boxes, test_size=0.2)

# Create a dataframe from the training data
train_df = pd.DataFrame({'filename': train_images, 'class': train_labels, 'xmin': [box[0] for box in train_boxes], 'ymin': [box[1] for box in train_boxes], 'xmax': [box[2] for box in train_boxes], 'ymax': [box[3] for box in train_boxes]})

# Create a dataframe from the validation data
val_df = pd.DataFrame({'filename': val_images, 'class': val_labels, 'xmin': [box[0] for box in val_boxes], 'ymin': [box[1] for box in val_boxes], 'xmax': [box[2] for box in val_boxes], 'ymax': [box[3] for box in val_boxes]})

# Define the train generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filename",
    y_col=["class", "xmin", "ymin", "xmax", "ymax"],
    target_size=(224, 224),
    batch_size=32,
    class_mode="raw",
    subset="training",
    shuffle=True,
    seed=42,
    dtype='float32')

# Define the validation generator
val_generator = train_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="filename",
    y_col=["class", "xmin", "ymin", "xmax", "ymax"],
    target_size=(224, 224),
    batch_size=32,
    class_mode="raw",   # set class_mode to "raw"
    subset="validation",
    shuffle=True,
    seed=42)


Found 1326 validated image filenames.
Found 83 validated image filenames.


In [39]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [43]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam

base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(4, activation='softmax', name='class_output')(x)

# Add custom bounding box regression layers
y = base_model.output
y = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(y)
y = layers.MaxPooling2D(pool_size=(2, 2))(y)
y = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(y)
y = layers.MaxPooling2D(pool_size=(2, 2))(y)
y = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(y)
y = layers.MaxPooling2D(pool_size=(1, 1))(y)
y = layers.Flatten()(y)
y = layers.Dense(512, activation='relu')(y)
y = layers.Dropout(0.5)(y)
y = layers.Dense(4, name='bbox_output')(y)


# Create a new model
model = Model(inputs=base_model.input, outputs=[predictions, y])

# Compile the model
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss=['categorical_crossentropy', 'mse'], loss_weights=[1., 1.])

# Define training loop with mixed precision
opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(optimizer)

# Train the model
epochs = 10
steps_per_epoch = len(train_generator)
validation_steps = len(val_generator)

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    
    # Train the model on one epoch of the training data
    model.fit(train_generator, steps_per_epoch=steps_per_epoch)
    
    # Evaluate the model on the validation data
    val_loss, val_acc = model.evaluate(val_generator, steps=validation_steps)
    print(f"Validation loss: {val_loss}, Validation accuracy: {val_acc}")
    
    # Save the model
    model.save(f"model_epoch{epoch+1}.h5")


Epoch 1/10


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [47]:
from sklearn.preprocessing import LabelEncoder

# Encode the 'class' column as numerical values
le = LabelEncoder()
train_df['class'] = le.fit_transform(train_df['class'])

# Define the train generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filename",
    y_col=["class", "xmin", "ymin", "xmax", "ymax"],
    target_size=(224, 224),
    batch_size=32,
    class_mode="raw",
    subset="training",
    shuffle=True,
    seed=42,
    dtype='float32')


Found 1326 validated image filenames.


In [48]:
# Train the model
epochs = 10
steps_per_epoch = len(train_generator)
validation_steps = len(val_generator)

In [53]:
# Define the base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
class_output = layers.Dense(4, activation='softmax', name='class_output')(x)

# Add custom bounding box regression layers
y = base_model.output
y = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(y)
y = layers.MaxPooling2D(pool_size=(2, 2))(y)
y = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(y)
y = layers.MaxPooling2D(pool_size=(2, 2))(y)
y = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(y)
y = layers.MaxPooling2D(pool_size=(1, 1))(y)
y = layers.Flatten()(y)
y = layers.Dense(512, activation='relu')(y)
y = layers.Dropout(0.5)(y)
bbox_output = layers.Dense(2, name='bbox_output', activation='sigmoid')(y)

# Create a new model
model = Model(inputs=base_model.input, outputs=[class_output, bbox_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'class_output': 'categorical_crossentropy', 'bbox_output': 'mse'},
              loss_weights={'class_output': 1., 'bbox_output': 1.})


In [58]:
data = next(train_generator)
print(data)


(array([[[[0.45751008, 0.45358852, 0.44574538],
         [0.4586301 , 0.44972447, 0.44021994],
         [0.40906692, 0.39338064, 0.38161594],
         ...,
         [0.5211879 , 0.50158006, 0.47805065],
         [0.5221539 , 0.5025461 , 0.47901666],
         [0.51408124, 0.4944734 , 0.47094396]],

        [[0.45751008, 0.45358852, 0.44574538],
         [0.4586301 , 0.44972447, 0.44021994],
         [0.40906692, 0.39338064, 0.38161594],
         ...,
         [0.5211879 , 0.50158006, 0.47805065],
         [0.5221539 , 0.5025461 , 0.47901666],
         [0.51408124, 0.4944734 , 0.47094396]],

        [[0.45751008, 0.45358852, 0.44574538],
         [0.4586301 , 0.44972447, 0.44021994],
         [0.40906692, 0.39338064, 0.38161594],
         ...,
         [0.5211879 , 0.50158006, 0.47805065],
         [0.5221539 , 0.5025461 , 0.47901666],
         [0.51408124, 0.4944734 , 0.47094396]],

        ...,

        [[0.59961927, 0.5623263 , 0.57393867],
         [0.58869904, 0.5528061 , 0.55174357

In [65]:
data = next(train_generator)
X = data[0]
Y_class = data[1][:, 0]
bbox = data[1][:, 1:]



In [72]:
label = data[1][:, 0]
bbox = data[1][:, 1:]


In [74]:
print(bbox)


[[ 49  94 423 364]
 [ 34 124 500 453]
 [ 71   4 458 383]
 [102 461 156 493]
 [178 233 234 326]
 [197 130 390 336]
 [328 280 382 320]
 [391  24 454  72]
 [337 237 512 486]
 [ 31 115 444 417]
 [ 73 158 171 310]
 [ 16  10 486 471]
 [ 24 200 254 482]
 [ 29 108 492 429]
 [ 93 172 411 417]
 [137  83 387 363]
 [ 35 104 499 397]
 [298 433 401 474]
 [126 384 256 512]
 [ 13  48 468 459]
 [312 270 341 293]
 [  3  30 512 510]
 [ 11 122 402 361]
 [253 227 315 289]
 [  0 320 429 496]
 [263 244 440 486]
 [  1 263 280 511]
 [ 10 141 503 382]
 [ 16 100 485 376]
 [ 24 129 450 387]
 [138 154 324 314]
 [344  98 465 275]]


In [79]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")

    # Train the model on one epoch of the training data
    data = next(train_generator)
    X = data[0]
    Y_class = data[1][:, 0]
    bbox = data[1][:, 1:]
    #Y_class = Y_class.astype('int')  # Convert the 'class' column to integer
    if any(x is None for x in [Y_class, bbox]):
        data = [x for x in [Y_class, bbox] if x is not None]
        model.fit(X, data, steps_per_epoch=steps_per_epoch)
    else:
        model.fit(X, [Y_class, bbox], steps_per_epoch=steps_per_epoch)


    # Evaluate the model on the validation data
    val_loss, val_acc = model.evaluate(val_generator, steps=validation_steps)
    print(f"Validation loss: {val_loss}, Validation accuracy: {val_acc}")

    # Save the model
    model.save(f"model_epoch{epoch+1}.h5")





Epoch 1/10


TypeError: 'NoneType' object is not callable

In [81]:
Y_class

array([0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0], dtype=int64)

In [80]:
bbox

array([[355, 159, 512, 405],
       [371, 382, 510, 512],
       [488, 107, 511, 141],
       [178,  40, 371, 456],
       [ 78,  74, 467, 442],
       [211, 347, 245, 376],
       [ 88, 169, 325, 474],
       [107, 174, 451, 405],
       [  4, 116, 343, 408],
       [172, 423, 202, 453],
       [  7,  19, 503, 487],
       [ 89, 236, 143, 277],
       [ 22,  60, 475, 382],
       [ 56, 126, 458, 364],
       [125, 180, 381, 422],
       [ 39, 100, 432, 374],
       [132, 100, 473, 438],
       [140, 170, 405, 378],
       [  8,  46, 462, 511],
       [ 42,  92, 476, 416],
       [  1,  50, 512, 507],
       [410, 269, 497, 363],
       [ 50,   3, 412, 512],
       [  2,  57, 443, 406],
       [ 73, 155, 376, 404],
       [ 99,  63, 439, 405],
       [ 25,  64, 386, 410],
       [111, 256, 171, 319],
       [ 36,  79, 419, 469],
       [  0,   1, 415, 439],
       [ 16, 448,  81, 499],
       [ 15,   5, 489, 477]], dtype=int64)